In [2]:
from decouple import Config, RepositoryEnv
from ollama_interaction import embeding_ollama_request,generate_ollama_request

import pandas as pd
import json

In [28]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

with open('model-data.json','r') as file:
    model_data = json.load(file)

for item in list(model_data.keys()):
    print('model :',item)

model : mistral-nemo:latest
model : mistral-small:latest
model : qwen2.5:14b
model : qwen2.5:32b
model : llama3.2:1b
model : llama3.1:latest


In [29]:
config = Config(RepositoryEnv('.env'))

ollama_api_url = config('OLLAMA_API_URL')
ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

In [9]:
data_question = pd.read_csv("hard-database/data_question.csv",index_col=0)

In [20]:
project_id = 0


project = data_question['project'][project_id]
question_list = data_question[['q1','q2','q3']].iloc[project_id].values

In [23]:
dumpy_answer_list = [
    """La revitalisation de la zone humide inclura des aménagements pour améliorer les circulations piétonnes, notamment à travers des sentiers et des passerelles permettant une meilleure accessibilité pour les visiteurs. Ces aménagements favoriseront également une meilleure gestion de la circulation routière aux abords de la zone, réduisant les risques pour les piétons et encourageant un accès sécurisé aux espaces naturels""",
    """Le projet est situé dans une commune comptant moins de 10 000 habitants, ce qui permet de bénéficier d’une aide prioritaire dans le cadre de cette subvention. La subvention d’investissement pourrait ainsi couvrir une part significative des coûts de revitalisation, renforçant les actions locales pour la préservation des écosystèmes et l’attractivité de la commune""",
    """La revitalisation de la zone humide s'inscrit dans les objectifs de la stratégie touristique départementale en valorisant les espaces naturels comme attraits pour les visiteurs. Ce projet vise à développer un écotourisme durable, en sensibilisant les visiteurs à la préservation des milieux naturels tout en renforçant l'attractivité locale grâce à des équipements respectueux de l'environnement et des parcours éducatifs pour les familles et les éco-touristes""",
    ]

In [41]:
def gen_prompt_project_summary(project: str, question_list: list[str], answer_list: list[str]) -> (str, str):
    prompt_system = """Tu es une IA bienveillante qui aide l'utilisateur à clarifier et synthétiser son projet.

    Ton rôle est de reformuler la description du projet en intégrant les réponses aux questions posées, pour aider l'utilisateur à obtenir une version plus détaillée et précise de son projet. 
    Attention : ne fais aucune mention de subventions, de financement, ou d'aides financières dans la reformulation."""
    
    # Génération du texte des questions-réponses
    question_answer_list_text = "\n".join(
        [f"- **Question {i + 1} :** {question_list[i]}\n  **Réponse :** {answer_list[i]}\n" for i in range(len(question_list))]
    )

    prompt_user = f"""**Instruction**

    Reformule la description du projet de manière concise et précise en tenant compte des réponses de l'utilisateur. La reformulation doit conserver le style initial du projet et rester courte, en une dizaine de paragraphes maximum (tu peux faire moins).

    - Ne fais aucune mention de subventions, de financement, ou d'aides financières.
    
    Voici les informations disponibles pour la reformulation :

    - **Description initiale du projet :** {project}
    - **Questions et réponses :**
    {question_answer_list_text}
    """

    return prompt_system, prompt_user


In [42]:
model = "qwen2.5:14b"
model_options = model_data[model]

request_options = {
    "num_ctx": 16384,
    "num_predict": 2048
}

num_question = 3

prompt_system, prompt_user = gen_prompt_project_summary(project,question_list,answer_list=dumpy_answer_list)

response = generate_ollama_request(
    prompt_system=prompt_system,
    response_format=None,
    prompt_user=prompt_user,
    ollama_api_url=ollama_api_url,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    model=model, 
    request_options= request_options,  # Default to None
    seed=0,
    )

In [43]:
print(response['response'])

### Description Révisée du Projet : Revitalisation d'une Zone Humide 

Le projet vise la revitalisation d’une zone humide située dans une commune de moins de 10.000 habitants, en mettant l'accent sur des aménagements qui favoriseront à la fois le développement et l'amélioration du tourisme local tout en préservant les écosystèmes naturels.

#### Améniations pour Circulations Piétonnes

La revitalisation comprendra plusieurs sentiers pédestres ainsi que de nouvelles passerelles. Ces aménagements permettront une meilleure accessibilité des visiteurs à l'intérieur et autour du site, tout en réduisant les risques routiers dans la zone environnante.

#### Gestion Améliorée

Les nouveaux équipements contribuerons également au meilleur accès sécurisé aux espaces naturels pour piétons. L’objectif est de créer une cohabitation harmonieuse entre l'usage des routes et les déplacements pédestres, en particulier dans le voisinage immédiate du site.

#### Objectifs Touristiques

Ce projet s'intègre 